In [1]:
"""
Pair Exercise 01: Approval System - Starter Code
=================================================
Build HITL approval for sensitive operations.

LEARNING GOALS:
- Import and configure HumanInTheLoopMiddleware
- Understand interrupt_on configuration
- Experience approval flow in LangSmith Studio
"""

import os
from dotenv import load_dotenv

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_core.tools import tool

load_dotenv()

model = init_chat_model("gpt-4o-mini", model_provider="openai")


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/Recature-Homework/Revature-Cognizant-Training-Exercises/January/venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
# =============================================================================
# TODO 2: Create Tools by Risk Level
# =============================================================================
# Create tools with different safety levels:
#
# Safe (no approval):
# - read_file: reads a file (simulated)
# - list_directory: lists directory contents (simulated)
#
# Moderate (requires approval):
# - write_file: writes to a file (simulated)
#
# Dangerous (definitely requires approval):
# - delete_file: deletes a file (simulated)
# - execute_command: runs a shell command (simulated)
#
# EXPERIMENT: What makes a good tool description for HITL?
# =============================================================================

@tool
def read_file(path: str) -> str:
    """Read contents of a file. Safe operation."""
    return f"[SIMULATED] Read file: {path}\nContent: This is simulated file content for {path}"

@tool
def list_directory(path: str) -> str:
    """List directory contents. Safe operation."""
    return f"[SIMULATED] Directory listing for {path}:\n- file1.txt\n- file2.log\n- subdirectory/"

@tool
def write_file(path: str, content: str) -> str:
    """Write to a file. MODERATE RISK - modifies filesystem."""
    return f"[SIMULATED] Successfully wrote {len(content)} characters to {path}"

@tool
def delete_file(path: str) -> str:
    """Delete a file. HIGH RISK - permanent data loss!"""
    return f"[SIMULATED] Successfully deleted file: {path}"

@tool
def execute_command(cmd: str) -> str:
    """Execute shell command. HIGH RISK - system access!"""
    return f"[SIMULATED] Executed command: {cmd}\nOutput: Command completed successfully"

In [3]:
# =============================================================================
# TODO 3: Configure HITL Middleware
# =============================================================================
# Create HumanInTheLoopMiddleware with:
# - interrupt_on: dict mapping tool names to True/False
#   - True = requires approval
#   - False = runs automatically
# - description_prefix: message shown in approval UI
#
# Configure so:
# - read_file, list_directory: no approval
# - write_file, delete_file, execute_command: require approval
#
# EXPERIMENT: What if all tools require approval?
# EXPERIMENT: Try different description_prefix messages
# =============================================================================

hitl_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "read_file": False,
        "list_directory": False,
        "write_file": True,
        "delete_file": True,
        "execute_command": True,
    },
    description_prefix="⚠️ APPROVAL REQUIRED: This operation requires human approval before execution."
)

In [4]:
# =============================================================================
# TODO 4: Create the Agent
# =============================================================================
# Create an agent that:
# - Uses all the file management tools
# - Uses your HITL middleware
# - Has a system prompt explaining risk levels
# - Does NOT include checkpointer (Studio handles it)
# =============================================================================

agent = create_agent(
    model=model,
    tools=[read_file, list_directory, write_file, delete_file, execute_command],
    middleware=[hitl_middleware],
    system_prompt="""You are a file management assistant with safety controls.

Available operations:
- SAFE (auto-approved): read_file, list_directory
- MODERATE RISK (requires approval): write_file
- HIGH RISK (requires approval): delete_file, execute_command

Always inform users about risk levels and explain what operations require approval.""",
    name="file_manager_agent"
)

In [5]:
# =============================================================================
# Testing - MUST USE LANGSMITH STUDIO
# =============================================================================
# This exercise MUST be tested in LangGraph Studio!
#
# STEPS TO CONNECT:
# 1. Install LangGraph CLI: pip install langgraph-cli
# 2. Navigate to this directory in terminal
# 3. Run: langgraph dev
# 4. Studio will open automatically in your browser
#
# Then in Studio, try:
# - 'List files in /home' -> should run immediately
# - 'Delete temp.log' -> should pause for approval
# - 'Write hello to greeting.txt' -> should pause for approval
#
# NOTE: A graph.py file has been created for Studio export
# =============================================================================

print("=" * 60)
print("CONNECTING TO LANGGRAPH STUDIO")
print("=" * 60)
print("\n1. Install LangGraph CLI:")
print("   pip install langgraph-cli")
print("\n2. Open terminal in this directory:")
print("   cd", os.getcwd())
print("\n3. Start Studio:")
print("   langgraph dev")
print("\n4. Studio will open automatically!")
print("\n" + "=" * 60)
print("\nFor notebook testing (limited - no approval UI):")
print("=" * 60)

from langgraph.checkpoint.memory import InMemorySaver

agent_with_checkpoint = create_agent(
    model=model,
    tools=[read_file, list_directory, write_file, delete_file, execute_command],
    middleware=[hitl_middleware],
    checkpointer=InMemorySaver(),
    system_prompt="""You are a file management assistant with safety controls.

Available operations:
- SAFE (auto-approved): read_file, list_directory
- MODERATE RISK (requires approval): write_file
- HIGH RISK (requires approval): delete_file, execute_command

Always inform users about risk levels and explain what operations require approval.""",
    name="file_manager_agent"
)

config = {"configurable": {"thread_id": "test_session"}}

test_prompt = "List files in /home"
print(f"\nTest: {test_prompt}")
result = agent_with_checkpoint.invoke({
    "messages": [{"role": "user", "content": test_prompt}]
}, config)
print(f"Result: {result['messages'][-1].content[:200]}...")
print("\n⚠️ NOTE: Approval UI only works in LangGraph Studio!")

This exercise MUST be tested in LangSmith Studio!

Run: cd exercises && langgraph dev

Then in Studio, try:
- 'List files in /home' -> should run immediately
- 'Delete temp.log' -> should pause for approval
- 'Write hello to greeting.txt' -> should pause for approval

Note: For notebook testing, use a checkpointer:

Test: List files in /home
Result: Here are the files and directories in `/home`:

- file1.txt
- file2.log
- subdirectory/...
